In [ ]:
# 데이터를 가져오는부분
from keras.datasets import boston_housing
(train_data,train_labels),(test_data,test_labels)=boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [ ]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [ ]:
print(train_data[0]) 
# 수치가 굉장히 폭이 넓음
# 특정 data의 영향력이 커짐->Scaling 필요
# column은 총 13개 

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [6]:
# 데이터 전처리 (Scaling) - standard 

mean = train_data.mean(axis=0) # 각 column의 평균치
train_data -= mean             # ex) 0번째 columns : train_data[][0]-train_data.mean[0] 
std = train_data.std(axis=0)   # 각 column의 표준편차
train_data /= std              # train_data=train_data/std

train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [8]:
# 모델의 구성 
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],))) # (train_data.shape[1], )와 같이 쉼표를 추가해 튜플임을 명시
  model.add(layers.Dense(64, activation='relu')) ,                                   # (train_data.shape[1])은 단순히 정수값으로 인식
  model.add(layers.Dense(1)) # 집값에 대한 unit이여서 1이고 회귀값은 activation 지정 안 함
  model.compile(optimizer='rmsprop',
                loss='mse',       
                metrics=['mse']) 
  return model

# k-fold validation
- 데이터의 양이 많지 않을때 사용함
- 데이터 편중을 막기 위해서 여러 세트로 구성된 학습과 평가 진행

In [9]:
from functools import partial
# K-folder 검증을 사용한 훈련 검증 

import numpy as np 

k = 4                                     # folder 개수
num_val_samples = len(train_data) // k    # 한 folder의 데이터 수
all_scores = []

for i in range(k):
  print('처리중인 폴드 #', i)

  # 검증 데이터의 준비
  val_data=train_data[i*num_val_samples : (i+1)*num_val_samples]          # 시작점: 끝점
  val_labels=train_labels[i*num_val_samples : (i+1)*num_val_samples]  
  
  # 학습 데이터의 준비
  data1=train_data[:i*num_val_samples]
  data2=train_data[(i+1)*num_val_samples:]
  data1_labels=train_labels[:i*num_val_samples]
  data2_labels=train_labels[(i+1)*num_val_samples:]
                                                                          # val data를 제외한 나머지 train data들을 합치기
  partial_train_data=np.concatenate([data1,data2],axis=0)                 # concatenate() : 2개의 (구조가 같은)데이터를 하나로 합쳐줌  
  partial_train_labels=np.concatenate([data1_labels,data2_labels],axis=0) #                 axis=0(columns 가 defualt)

  # 모델의 학습
  model = build_model()
  model.summary()

  model.fit(partial_train_data, partial_train_labels, epochs=500, batch_size=128, verbose=0) # verbose=0 : 훈련사항을 보여주지 않겠다. (default=1)

  # 모델의 검증
  val_mse,val_mae=model.evaluate(val_data,val_labels)
  print(val_mse,val_mae)
  all_scores.append(val_mae)

처리중인 폴드 # 0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 6ms/step - loss: 7.8834 - mse: 7.8834
7.883386135101318 7.883386135101318
처리중인 폴드 # 1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)        

In [11]:
np.mean(all_scores)

10.28221046924591